In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from statsmodels.graphics import tsaplots
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA, ARIMAResults
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.stats.diagnostic import acorr_ljungbox
from scipy import signal

import os
import itertools

In [ ]:
# plt.style.available

In [ ]:
# plt.rcParams.keys()

In [ ]:
# sns.heatmap(correlation_matrix,
#             annot=True,
#             linewidths=0.4,
#             annot_kws={"size": 10})

### import data

In [ ]:
# appointments = pd.read_csv('Appointments.csv')
appointments = pd.read_csv('AppointmentsSince2015.csv')

In [ ]:
calls = pd.read_csv('CallsRingCentral.csv')

In [ ]:
reason_for_visit = pd.read_csv('MeetingReasonForVisits.csv')

In [ ]:
meeting_status = pd.read_csv('MeetingStatus.csv')

In [ ]:
offices = pd.read_csv('Offices.csv')

In [ ]:
providers_schedules = pd.read_csv('ProvidersSchedulesLastest.csv')

### explore data

In [ ]:
appointments.info()

In [ ]:
appointments.head().T

In [ ]:
reason_for_visit.head()

In [ ]:
reason_for_visit.info()

In [ ]:
len(reason_for_visit['Name'].unique())

In [ ]:
meeting_status.head()

In [ ]:
meeting_status.info()

In [ ]:
offices.head()

In [ ]:
offices.info()

In [ ]:
providers_schedules.head()

In [ ]:
providers_schedules.info()

In [ ]:
calls.head().T

In [ ]:
calls.info()

### combine/merge dataframes

In [ ]:
appointments['Specialty'].unique()

In [ ]:
appointments.columns

In [ ]:
pd.value_counts(appointments['Specialty'])

In [ ]:
doctors = ['Psychiatry', 'Child & Adolescent Psychiatry', ]
RN_PAs = ['Medical', 'Psych/Mental Health, Child & Adolescent', 'Psych/Mental Health', 'Physician Assistant']
therapists = ['Marriage & Family Therapist', 'Psychologist', 'Specialist/Technologist, Other', 'Clinical' ]

In [ ]:
appointments['Specialty'].loc[appointments['Specialty'].isin(doctors)]= 'doctor'
appointments['Specialty'].loc[appointments['Specialty'].isin(RN_PAs)] = 'RN/PA'
appointments['Specialty'].loc[appointments['Specialty'].isin(therapists)] = 'therapist'

In [ ]:
pd.value_counts(appointments['Specialty'])

In [ ]:
appointments['Specialty'].isnull().sum()

In [ ]:
merged1 = pd.merge(left=appointments, right=reason_for_visit, how='left', left_on='MeetingReasonForVisitId',\
                  right_on='Id')

In [ ]:
merged1 = merged1.rename(columns={'MeetingReasonForVisitId': 'ReasonForVisitId', 'Name':'ReasonForVisitName', 'Description':'ReasonForVisitDescription'})

In [ ]:
merged1.columns

In [ ]:
merged1.drop('Id', axis=1, inplace=True)

In [ ]:
# merge in office name from offices df
merged1 = pd.merge(left=merged1, right=offices, how='left', left_on='OfficeId', right_on='id')

In [ ]:
merged1 = merged1.rename(columns={'Name':'OfficeName', 'id_x':'id'})

In [ ]:
merged1.drop('id_y', axis=1, inplace=True)

In [ ]:
merged1 = pd.merge(left=merged1, right=meeting_status, how='left', left_on='MeetingStatusId', right_on='Id')

In [ ]:
merged1 = merged1.rename(columns={'Name':'MeetingStatusName', 'Description':'MeetingStatusDescription'})

In [ ]:
merged1.drop('Id', axis=1, inplace=True)

In [ ]:
merged1.info()

In [ ]:
# merged1.head().T

In [ ]:
# rearrange column order to group releveant columns together
merged1.columns

In [ ]:
# reorder columns within the df
ordered_columns = ['id', 'Patient', 'PatientAgeMeetingDate', 'PatientGender',
       'PatientState', 'PatientCity', 'PatientInsurance', 'Provider',
       'Specialty', 'AppointmentDate', 'AppointmentCreated', 'AppointmentDuration', 'ReasonForVisitId', 'ReasonForVisitName',
       'ReasonForVisitDescription','MeetingStatusId', 'MeetingStatusName',
       'MeetingStatusDescription', 'OfficeId',  'OfficeName', 'CreatedBy']

In [ ]:
merged1 = merged1[ordered_columns]

In [ ]:
# id any missing specialties
merged1['Specialty'].isnull().sum(), merged1['Specialty'].notnull().sum()

### Data Cleaning: 
#### filling NaN values

In [ ]:
no_specialty = merged1[appointments['Specialty'].isnull()] 

In [ ]:
no_specialty = no_specialty[['Provider', 'Specialty', 'AppointmentDate', 'AppointmentCreated',\
        'AppointmentDuration', 'ReasonForVisitId', 'ReasonForVisitName',
       'ReasonForVisitDescription','MeetingStatusId', 'MeetingStatusName', 'MeetingStatusDescription', \
    'OfficeId',  'OfficeName']]

In [ ]:
# no_specialty.head().T

In [ ]:
pd.value_counts(no_specialty['ReasonForVisitName'])

In [ ]:
# filling NaN values in Specialty
implied_therapy = ['Therapy', 'New Patient Therapy', ]
implied_doctor = ['Therapy Telepsychiatry','Follow up Telepsychiatry', 'New Patient Therapy Telepsychiatry',\
                  'New Patient MD Adult', 'New Patient MD Adult Telepsychiatry']
merged1['Specialty'].loc[merged1['ReasonForVisitName'].isin(implied_therapy)] = 'therapist'
merged1['Specialty'].loc[merged1['ReasonForVisitName'].isin(implied_doctor)] = 'doctor'

In [ ]:
# most missing values in Specialty are now filled
merged1['Specialty'].isnull().sum(), merged1['Specialty'].notnull().sum()

In [ ]:
# # remove time component from AppointmentDate and AppointmentCreated columns
# merged1['AppointmentCreated'] = merged1['AppointmentCreated'].str.slice(start=0, stop=10)
# merged1['AppointmentDate'] = merged1['AppointmentDate'].str.slice(start=0, stop=10)

In [ ]:
# pd.value_counts(merged1['PatientState']), pd.value_counts(merged1['PatientCity'])

In [ ]:
# pd.value_counts(merged1['PatientGender']), # pd.value_counts(merged1['MeetingStatusName'])

In [ ]:
pd.value_counts(merged1['AppointmentDuration'])

In [ ]:
pd.value_counts(merged1[merged1['AppointmentDuration'] > 90]['ReasonForVisitName'])

In [ ]:
# convert date columns to datetime 
merged1['AppointmentCreated'] = pd.to_datetime(merged1['AppointmentCreated'], errors='coerce')#.apply(lambda x: x.date()) #, format='%Y-%m-%d')
merged1['AppointmentDate'] = pd.to_datetime(merged1['AppointmentDate'], errors='coerce')#.apply(lambda x: x.date()) #, format='%Y-%m-%d')

In [ ]:
# calculate time between AppointmentCreated and AppointmentDate
merged1['DaysFromAppointmentCreatedToVisit'] = (merged1['AppointmentDate'] - merged1['AppointmentCreated']).dt.days

In [ ]:
# merged1.info()

In [ ]:
# merged1['Specialty'].isnull()
merged1.isnull().sum()

In [ ]:
# pd.value_counts(merged1['DaysFromAppointmentCreatedToVisit'])

In [ ]:
# columns where time between when appointment was created and appointment was negative
# merged1[merged1['DaysFromAppointmentCreatedToVisit'] < 0][['DaysFromAppointmentCreatedToVisit', 'AppointmentCreated', 'AppointmentDate']]

In [ ]:
# merged1.dtypes

In [ ]:
pd.value_counts(merged1['Specialty'])

In [ ]:
merged1 = merged1.set_index('AppointmentDate')

In [ ]:
merged2 = merged1.copy()

In [ ]:
merged2.index

In [ ]:
merged2 = merged2['2018-02-28':]

In [ ]:
test_data = merged1['2018-04-30':'2018-02-28']

In [ ]:
# merged2.info()

In [ ]:
# drop rows with missing specialty
merged2.dropna(subset=['Specialty'], how='all', inplace=True)

In [ ]:
merged_index_month = merged2.index.month

In [ ]:
merged_index_year = merged2.index.year

In [ ]:
merged2.index.date

In [ ]:
def plot_num_appointments_by_month(df, group_col, plot_name, colormap='Dark2'):
    """ Plot duration data grouped by month"""
    ax = df.groupby([df.index.year, df.index.month, group_col])[group_col]\
    .count().unstack().plot(figsize=(10,8),colormap=colormap, linewidth=3, fontsize=12, rot=30)
    ax.set_title(plot_name)
    ax.set_xlabel('Date')
    ax.set_ylabel('Number of Appointments')
    ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5), fontsize=12)
    plt.show()

In [ ]:
# plot number of appoints by specialty for each month through February 2018, adding multiple years
plot_num_appointments_by_month(df=merged2, group_col='Specialty', plot_name=\
        'Number of Appointments per Month by Specialty', colormap='Dark2')

In [ ]:
merged2['DurationHours'] = merged2['AppointmentDuration'] /60

In [ ]:
time_spent_by_month = merged2.groupby([merged_index_year, merged_index_month, 'Specialty'])['DurationHours'].sum()

In [ ]:
time_spent_by_month.head()

In [ ]:
def plot_time_spent_by_month(df, group_col, duration_col, plot_name, colormap='Dark2'):
    """ Plot duration data grouped by month"""
    ax = df.groupby([df.index.year, df.index.month, group_col])[duration_col]\
    .sum().unstack().plot(figsize=(10,8),colormap=colormap, linewidth=3, fontsize=12, rot=30)
    ax.set_title(plot_name)
    ax.set_xlabel('Date')
    ax.set_ylabel('Time')
    ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5), fontsize=12)
    plt.show()

In [ ]:
plot_time_spent_by_month(df=merged2, group_col='Specialty', duration_col='DurationHours', \
            plot_name='Time Spent by Specialty (hours)', colormap='Dark2')

In [ ]:
# date = merged2.index
# merged2.groupby([date,'Specialty'])['DurationHours'].sum()

In [ ]:
date = pd.to_datetime(merged2.index,format='%Y-%m-%d')

In [ ]:
merged2['AppointmentDate'] = date

In [ ]:
def plot_time_spent(df, date_col, group_col, duration_col, plot_name, colormap='Dark2'):
    """ Plot duration data grouped by month"""
    ax = df.groupby([date_col, group_col])[duration_col]\
    .sum().unstack().plot(figsize=(10,8),colormap=colormap, linewidth=3, fontsize=12, rot=30)
    ax.set_title(plot_name)
    ax.set_xlabel('Date')
#     ax.set_ylabel('Time')
    ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5), fontsize=12)
    plt.show()

In [ ]:
plot_time_spent(df=merged2, date_col='AppointmentDate', group_col='Specialty', duration_col='DurationHours',\
            plot_name='Time per Month by Specialty (hours)', colormap='Dark2')

In [ ]:
# params = {'legend.fontsize': 20, 'legend.handlelength': 2, 'axes.labelsize': 'medium'}
# plt.rcParams.update(params)
# fig = merged2.groupby(['AppointmentDate', 'Specialty'])['DurationHours'].sum().unstack().plot.area(subplots=True,\
#     colormap='Dark2', figsize=(20,50), layout=(10,1), sharex=True, sharey=True, linewidth=3, fontsize=20)
# plt.xlabel('Date')
# plt.ylabel('Time (hours)', fontsize=20)
# plt.tight_layout()
# plt.show()

In [ ]:
merged2['AppointmentDate'] = merged2.index

In [ ]:
duration_df = merged2[['Provider', 'Specialty', 'AppointmentCreated', 'AppointmentDate', 'AppointmentDuration',
       'ReasonForVisitName', 'DurationHours', 'ReasonForVisitDescription','MeetingStatusName', 'MeetingStatusDescription',
       'OfficeId']]

In [ ]:
duration_df.info()

In [ ]:
duration_df.isnull().sum()

In [ ]:
# drop appointments that are longer than 90 minutes
duration_df = duration_df[duration_df['AppointmentDuration'] <= 90]

In [ ]:
# drop remaining columns with missing values
duration_df.dropna(axis=0, inplace=True)

In [ ]:
# params = {'legend.fontsize': 20, 'legend.handlelength': 2, 'axes.labelsize': 'medium'}
# plt.rcParams.update(params)
# fig2 = duration_df.groupby(['AppointmentDate', 'Specialty'])['AppointmentDuration'].sum().unstack().plot.area(\
#     colormap='Dark2', figsize=(40,20), linewidth=3, fontsize=20)
# plt.xlabel('Date')
# plt.ylabel('Time (minutes)', fontsize=20)
# plt.tight_layout()
# plt.show()

In [ ]:
# params = {'legend.fontsize': 20, 'legend.handlelength': 2, 'axes.labelsize': 'medium'}
# plt.rcParams.update(params)
# fig2 = duration_df.groupby(['AppointmentDate', 'Specialty'])['DurationHours'].sum().unstack().plot.area(subplots=True,\
#     colormap='Dark2', figsize=(20,60), layout=(10,1), sharex=True, sharey=True, linewidth=3, fontsize=20)
# plt.xlabel('Date')
# plt.ylabel('Time (minutes)', fontsize=20)
# plt.tight_layout()
# plt.show()

In [ ]:
doctors = duration_df[duration_df['Specialty'] == 'doctor']
therapists = duration_df[duration_df['Specialty'] == 'therapist']
RN_PA = duration_df[duration_df['Specialty'] == 'RN/PA']

In [ ]:
doc_fig = doctors.groupby([doctors.index.year, doctors.index.month])['DurationHours'].sum().plot(\
        figsize=(8,8), fontsize=10, rot=30)
                              
plt.title('Doctors', fontsize=15)
plt.xlabel('Date')
plt.ylabel('Time (hours)', fontsize=12)
plt.show()

In [ ]:
doc_duration = doctors.groupby(doctors.index.date)['DurationHours'].sum()
RN_PA_duration = RN_PA.groupby(RN_PA.index.date)['DurationHours'].sum()
therapist_duration = therapists.groupby(therapists.index.date)['DurationHours'].sum()

In [ ]:
def plot_series(series, xlabel, ylabel, plot_name):
    "Plots simple time series from Pandas Series"
    ax = series.plot(figsize=(8,3), linewidth = 3, fontsize=10, grid=True, rot=30)
    ax.set_title(plot_name, fontsize=18)
    ax.set_xlabel(xlabel, fontsize=15)
    ax.set_ylabel(ylabel, fontsize=15)
    plt.show()

In [ ]:
# plot_series(series=doc_duration, xlabel='Date', ylabel='Time Spent (hours)', plot_name='Doctors Series')

In [ ]:
def plot_series_and_differences(series, ax, num_diff, title):
    "Plot raw data and specified number of differences"
    ax[0].plot(series.index, series)
    ax[0].set_title('Raw series: {}'.format(title))
    for i in range(1, num_diff+1):
        diff = series.diff(i)
        ax[i].plot(series.index, diff)
        ax[i].set_title('Difference # {}'.format(str(i)))   

In [ ]:
fig, axes = plt.subplots(3, figsize=(10, 8))
plot_series_and_differences(series=doc_duration, ax=axes, num_diff=2, title='Doctors')
fig.tight_layout()

In [ ]:
def run_augmented_Dickey_Fuller_test(series, num_diffs=None):
    test = sm.tsa.stattools.adfuller(series)
    if test[1] >= 0.05:
        print('The p-value for the series is: {p}, which is not significant'.format(p=test[1]))
    else:
        print('The p-value for the series is: {p}, which is significant'.format(p=test[1]))  
    if num_diffs:
        for i in range(1, num_diffs +1):
            test = sm.tsa.stattools.adfuller(series.diff(i)[i:])
            if test[1] >= 0.05:
                print('The p-value for difference {diff} is: {p}, which is not significant'.format(diff=str(i), p=test[1]))
            else:
                print('The p-value for difference {diff} is: {p}, which is significant'.format(diff=str(i), p=test[1]))   

In [ ]:
# test for stationarity of doctors data, 1st and 2nd diff
run_augmented_Dickey_Fuller_test(doc_duration, num_diffs=2)

In [ ]:
fig, axes = plt.subplots(3, figsize=(10, 8))
plot_series_and_differences(series=RN_PA_duration, ax=axes, num_diff=2, title='RN/PA')
fig.tight_layout()

In [ ]:
# test for stationarity of RN/PA data, 1st and 2nd diff
run_augmented_Dickey_Fuller_test(RN_PA_duration, num_diffs=2)

In [ ]:
fig, axes = plt.subplots(3, figsize=(10, 8))
plot_series_and_differences(series=therapist_duration, ax=axes, num_diff=2, title='Therapists')
fig.tight_layout()

In [ ]:
# test for stationarity of therapist data, 1st and 2nd diff
run_augmented_Dickey_Fuller_test(therapist_duration, num_diffs=2)

In [ ]:
## plot detrended doc_dur using functions from matt drury tine series lecture w/ some 
# modifications
def make_col_vector(array):
    """Convert a one dimensional numpy array to a column vector."""
    return array.reshape(-1, 1)

def make_design_matrix(array):
    """Construct a design matrix from a numpy array, including an intercept term."""
    return sm.add_constant(make_col_vector(array), prepend=False)

In [ ]:
def fit_linear_trend(series):
    """Fit a linear trend to a time series.  Return the fit trend as a numpy array."""
    X = make_design_matrix(np.arange(len(series)) + 1)
    linear_trend_ols = sm.OLS(series.values, X).fit()
    linear_trend = linear_trend_ols.predict(X)
    return linear_trend

In [ ]:
def plot_trend_data(ax, name, series):
    ax.plot(series.index, series)
    
def plot_linear_trend(ax, name, series):
    linear_trend = fit_linear_trend(series)
    plot_trend_data(ax, name, series)
    ax.plot(series.index, linear_trend)
    ax.set_title(name)

In [ ]:
# plot linear model for doctors data
fig, ax = plt.subplots(1, figsize=(10,3))
plot_linear_trend(ax, 'Linear Trend Doctors', doc_duration)
plt.tight_layout()

In [ ]:
# get detrended series by subtracting the linear fit trend from original data
doctors_trend = fit_linear_trend(doc_duration)
doctors_detrended = doc_duration - doctors_trend

In [ ]:
# plot detrended data
fig, ax = plt.subplots(1, figsize=(10,3))
plot_linear_trend(ax, 'Doctors', doctors_detrended)
plt.title('Doctors data, linearly detrended')
plt.tight_layout()

In [ ]:
# Calculate and plot moving average
def fit_moving_average_trend(series, window=14):
    return series.rolling(window, center=True).mean()

def plot_with_moving_average(ax, name, series, window=6):
    moving_average_trend = fit_moving_average_trend(series, window)
    plot_trend_data(ax, name, series)
    ax.plot(series.index, moving_average_trend)
    ax.set_title('{title}, window={w}'.format(title=name, w=str(window)))

In [ ]:
fig, ax = plt.subplots(1, figsize=(10,3))
plot_with_moving_average(ax, 'Moving AVG Doctors', doc_duration)

In [ ]:
fig, ax = plt.subplots(1, figsize=(10,3))
plot_with_moving_average(ax, 'Moving AVG Doctors', doc_duration, window=31)

In [ ]:
# look for seasonal patterns using window=52
fig, ax = plt.subplots(1, figsize=(10,3))
plot_with_moving_average(ax, 'Seasonal AVG Doctors', doc_duration, window=52)

In [ ]:
# # Plot moving average
# doc_mean = pd.rolling_mean(doc_duration, window=30)
# ax = doc_mean.plot(figsize=(10,6), linewidth=2, fontsize=12)
# ax.set_title('30 day rolling mean of doc duration', fontsize=18)
# ax.set_xlabel('Date', fontsize=15)
# plt.show()

In [ ]:
def plot_autocorrelation(series, params, lags, alpha, title):
    plt.rcParams.update(params)
    acf_plot = tsaplots.plot_acf(series, lags=lags, alpha=alpha)
    plt.title(title)
    plt.show()

def plot_partial_autocorrelation(series, params, lags, alpha, title):
    plt.rcParams.update(params)
    acf_plot = tsaplots.plot_pacf(series, lags=lags, alpha=alpha)
    plt.title(title)
    plt.show()

In [ ]:
params = {'figure.figsize': [6,6],'axes.labelsize': 'large', 'font.size': 10.0, 'lines.linewidth': 3}
plot_autocorrelation(doc_duration, params=params, lags=31, alpha=0.05, title='Doctors Time Spent Autocorrelation')

In [ ]:
params = {'figure.figsize': [6,6],'axes.labelsize': 'large', 'font.size': 10.0, 'lines.linewidth': 3}
plot_partial_autocorrelation(doc_duration, params=params, lags=31, alpha=0.05, \
                             title='Doctors Time Spent Partial Autocorrelation')

In [ ]:
d_ts_index = pd.to_datetime(doc_duration.index)
RN_ts_index = pd.to_datetime(RN_PA_duration.index)
t_ts_index = pd.to_datetime(therapist_duration.index)

In [ ]:
doc_duration.index = d_ts_index
RN_PA_duration.index = RN_ts_index
therapist_duration.index = t_ts_index

In [ ]:
def plot_decomposition(series, params, freq, title):
    "Plots observed, trend, seasonal, residual"
    plt.rcParams.update(params)
    decomp = sm.tsa.seasonal_decompose(series, freq=freq)
    fig = decomp.plot()
    plt.title(title)
    plt.show()

In [ ]:
params = {'figure.figsize': [8, 8],'axes.labelsize': 'Medium', 'font.size': 12.0, 'lines.linewidth': 2}
plot_decomposition(doc_duration, params=params, freq=31, title='Doctors Decomposition')

In [ ]:
params = {'figure.figsize': [8,8],'axes.labelsize': 'Medium', 'font.size': 12.0, 'lines.linewidth': 2}
plot_decomposition(RN_PA_duration, params=params, freq=31, title='RN/PA Decomposition')

In [ ]:
params = {'figure.figsize': [8,8],'axes.labelsize': 'Medium', 'font.size': 12.0, 'lines.linewidth': 2}
plot_decomposition(therapist_duration, params=params, freq=31, title='Therapist Decomposition')

In [ ]:
plt.rcParams.keys()

In [ ]:
# index_year = df.index.year
# df_by_year = df.groupby(index_year).mean()

In [ ]:
# # downsample from daily to weekly & montly data
# weekly_doc_dur = doc_duration.resample(rule='W').last() # weekly time spent

# ax = weekly_doc_dur.plot(figsize=(10,6), linewidth = 3, fontsize=10, grid=True)
# ax.set_title('Doctors Time Spent Weekly', fontsize=18)
# ax.set_xlabel('Date', fontsize=15)
# ax.set_ylabel('Time (hours)', fontsize=15)
# plt.show()